# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

In [4]:
len(train_doc)
len(val_doc)

100

## Preprocessing

In [5]:
# Helper Functions and our current preprogessing pipeline
# pre process ===> conbert_to_grayscale --> normalize_data

# Convert to grayscale
def convert_to_grayscale(data):
    #return np.sum(data/3, axis=3, keepdims=True)
    return data.mean(axis=-1,keepdims=1) 

def normalize_data(data):
    return data/127.5-1

def preprocess_pipeline(data):
    return normalize_data(convert_to_grayscale(data))

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
c = 3
    
def generator(source_path, folder_list, batch_size):
    
    x = 30 # number of frames
    y = 120 # image width
    z = 120 # image height
    
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,c)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = normalize_data(temp)
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
       
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,c)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = normalize_data(temp)
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
        

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
input_shape = (30, 120, 120, c)

In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

model_a = Sequential()

model_a.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(nb_dense[0], activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(nb_dense[1], activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(nb_dense[2], activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [10]:
#rate = 0.00097
rate = 0.001
optimiser = optimizers.Adam(lr=rate) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:
model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
Source path =  ./Project_data/train ; batch size = 10
Source path =  ./Project_data/val ; batch size = 10


/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 0s - loss: 6.2914 - categorical_accuracy: 0.2833

/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 46s 687ms/step - loss: 6.2396 - categorical_accuracy: 0.2791 - val_loss: 4.1348 - val_categorical_accuracy: 0.3700

Epoch 00001: saving model to model_init_2018-10-2007_29_22.855379/model-00001-6.27555-0.28205-4.13475-0.37000.h5
Epoch 2/10
67/67 [==============================] - 15s 217ms/step - loss: 3.6477 - categorical_accuracy: 0.2090 - val_loss: 2.2293 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2018-10-2007_29_22.855379/model-00002-3.64774-0.20896-2.22925-0.22000.h5
Epoch 3/10
67/67 [==============================] - 16s 243ms/step - loss: 1.9255 - categorical_accuracy: 0.3483 - val_loss: 1.2189 - val_categorical_accuracy: 0.4300

Epoch 00003: saving model to model_init_2018-10-2007_29_22.855379/model-00003-1.92554-0.34826-1.21889-0.43000.h5
Epoch 4/10
67/67 [==============================] - 14s 208ms/step - loss: 1.6871 - categorical_accuracy: 0.3383 - val_loss: 1.2734 - val_categorical_accuracy: 0.4400



In [73]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

## New Generator

In [74]:
def new_generator(source_path, folder_list, batch_size):
      
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,nb_frames)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(nb_rows,nb_cols))
                    #temp = normalize_data(temp)
                    temp = preprocess_pipeline(temp)
                    
                    batch_data[folder,idx] = temp

       
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(nb_rows,nb_cols))
                    #temp = normalize_data(temp)
                    temp = preprocess_pipeline(temp)
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
        

In [62]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

# Define model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(nb_frames,nb_rows,nb_cols,nb_channel), padding='same'))
model.add(Activation('relu'))
model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_61 (Conv3D)           (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_61 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_62 (Conv3D)           (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_62 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_33 (MaxPooling (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_45 (Dropout)         (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_63 (Conv3D)           (None, 10, 40, 40, 64)    55360     
__________

In [63]:
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)

In [64]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10


/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 1s - loss: 1.7075 - categorical_accuracy: 0.1938Batch:  67 Index: 10


/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 49s 725ms/step - loss: 1.7046 - categorical_accuracy: 0.1911 - val_loss: 1.6084 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2018-10-2007_29_22.855379/model-00001-1.70556-0.19306-1.60844-0.23000.h5
Epoch 2/10
67/67 [==============================] - 15s 228ms/step - loss: 1.6091 - categorical_accuracy: 0.2040 - val_loss: 1.6064 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2018-10-2007_29_22.855379/model-00002-1.60910-0.20398-1.60639-0.23000.h5
Epoch 3/10
67/67 [==============================] - 16s 243ms/step - loss: 1.6104 - categorical_accuracy: 0.1940 - val_loss: 1.4960 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2018-10-2007_29_22.855379/model-00003-1.61044-0.19403-1.49597-0.23000.h5
Epoch 4/10
67/67 [==============================] - 16s 239ms/step - loss: 1.5694 - categorical_accuracy: 0.2289 - val_loss: 1.4670 - val_categorical_accuracy: 0.3000



In [65]:
# Define model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(nb_frames,nb_rows,nb_cols,nb_channel), padding="same"))
model.add(Activation('relu'))
model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_65 (Conv3D)           (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_65 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_66 (Conv3D)           (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_66 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_35 (MaxPooling (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_48 (Dropout)         (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_67 (Conv3D)           (None, 10, 40, 40, 64)    55360     
__________

In [66]:
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)

In [67]:
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
Source path =  ./Project_data/train ; batch size = 10
Source path =  ./Project_data/val ; batch size = 10


/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 1s - loss: 2.0563 - categorical_accuracy: 0.2723Batch:  67 Index: 10


/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 48s 723ms/step - loss: 2.0599 - categorical_accuracy: 0.2687 - val_loss: 5.0915 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2018-10-2007_29_22.855379/model-00001-2.04827-0.27149-5.09154-0.23000.h5
Epoch 2/10
67/67 [==============================] - 15s 231ms/step - loss: 1.8005 - categorical_accuracy: 0.3383 - val_loss: 4.6516 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2018-10-2007_29_22.855379/model-00002-1.80049-0.33831-4.65156-0.23000.h5
Epoch 3/10
67/67 [==============================] - 17s 249ms/step - loss: 1.9274 - categorical_accuracy: 0.2537 - val_loss: 1.4311 - val_categorical_accuracy: 0.4200

Epoch 00003: saving model to model_init_2018-10-2007_29_22.855379/model-00003-1.92736-0.25373-1.43114-0.42000.h5
Epoch 4/10
67/67 [==============================] - 15s 225ms/step - loss: 1.6868 - categorical_accuracy: 0.3383 - val_loss: 1.8842 - val_categorical_accuracy: 0.2400



In [68]:
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_71 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_71 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_38 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_72 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_72 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [69]:
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10


/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 0s - loss: 5.1125 - categorical_accuracy: 0.2682

/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 47s 695ms/step - loss: 5.0848 - categorical_accuracy: 0.2691 - val_loss: 5.6818 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2018-10-2007_29_22.855379/model-00001-5.10398-0.26848-5.68178-0.31000.h5
Epoch 2/20
67/67 [==============================] - 14s 210ms/step - loss: 2.3025 - categorical_accuracy: 0.3134 - val_loss: 1.6048 - val_categorical_accuracy: 0.3500

Epoch 00002: saving model to model_init_2018-10-2007_29_22.855379/model-00002-2.30247-0.31343-1.60477-0.35000.h5
Epoch 3/20
67/67 [==============================] - 16s 240ms/step - loss: 1.6830 - categorical_accuracy: 0.3582 - val_loss: 1.1917 - val_categorical_accuracy: 0.5200

Epoch 00003: saving model to model_init_2018-10-2007_29_22.855379/model-00003-1.68301-0.35821-1.19174-0.52000.h5
Epoch 4/20
67/67 [==============================] - 16s 242ms/step - loss: 1.4793 - categorical_accuracy: 0.3682 - val_loss: 1.4588 - val_categorical_accuracy: 0.2700



In [76]:
nb_frames = 30 # number of frames
nb_rows = 64 # image width
nb_cols = 64 # image height 

nb_classes = 5
nb_channel = 1

In [71]:
# Define model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(nb_frames,nb_rows,nb_cols,nb_channel), padding="same"))
model.add(Activation('relu'))
model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_75 (Conv3D)           (None, 30, 64, 64, 32)    896       
_________________________________________________________________
activation_75 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
conv3d_76 (Conv3D)           (None, 30, 64, 64, 32)    27680     
_________________________________________________________________
activation_76 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
max_pooling3d_42 (MaxPooling (None, 10, 22, 22, 32)    0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 10, 22, 22, 32)    0         
_________________________________________________________________
conv3d_77 (Conv3D)           (None, 10, 22, 22, 64)    55360     
__________

In [79]:
batch_size = 5
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  ./Project_data/train ; batch size = 5
Source path =  ./Project_data/val ; batch size = 5


/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 19s 290ms/step - loss: 1.6940 - categorical_accuracy: 0.2687 - val_loss: 2.0147 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2018-10-2007_29_22.855379/model-00001-1.69395-0.26866-2.01474-0.20000.h5
Epoch 2/20
66/67 [============================>.] - ETA: 0s - loss: 1.6394 - categorical_accuracy: 0.3212

/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 20s 296ms/step - loss: 1.6489 - categorical_accuracy: 0.3165 - val_loss: 1.5015 - val_categorical_accuracy: 0.3200

Epoch 00002: saving model to model_init_2018-10-2007_29_22.855379/model-00002-1.64084-0.32024-1.50153-0.32000.h5
Epoch 3/20
67/67 [==============================] - 12s 181ms/step - loss: 1.6278 - categorical_accuracy: 0.2886 - val_loss: 1.5605 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2018-10-2007_29_22.855379/model-00003-1.62783-0.28856-1.56051-0.30000.h5
Epoch 4/20
67/67 [==============================] - 13s 188ms/step - loss: 1.6503 - categorical_accuracy: 0.2637 - val_loss: 1.4302 - val_categorical_accuracy: 0.3800

Epoch 00004: saving model to model_init_2018-10-2007_29_22.855379/model-00004-1.65028-0.26368-1.43023-0.38000.h5
Epoch 5/20
67/67 [==============================] - 13s 191ms/step - loss: 1.7246 - categorical_accuracy: 0.2587 - val_loss: 1.5076 - val_categorical_accuracy: 0.3600



In [82]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 1

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_85 (Conv3D)           (None, 30, 120, 120, 8)   224       
_________________________________________________________________
batch_normalization_15 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_85 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_49 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_86 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_16 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_86 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [85]:
batch_size = 10
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
Source path =  ./Project_data/train ; batch size = 10
Source path =  ./Project_data/val ; batch size = 10


/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 0s - loss: 0.9877 - categorical_accuracy: 0.5864

/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/santhosh/anaconda3/envs/tensor36/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 45s 664ms/step - loss: 0.9844 - categorical_accuracy: 0.5925 - val_loss: 0.8864 - val_categorical_accuracy: 0.6900

Epoch 00001: saving model to model_init_2018-10-2007_29_22.855379/model-00001-0.98671-0.58824-0.88639-0.69000.h5
Epoch 2/30
67/67 [==============================] - 16s 246ms/step - loss: 1.0896 - categorical_accuracy: 0.5124 - val_loss: 0.8828 - val_categorical_accuracy: 0.6900

Epoch 00002: saving model to model_init_2018-10-2007_29_22.855379/model-00002-1.08958-0.51244-0.88280-0.69000.h5
Epoch 3/30
67/67 [==============================] - 16s 232ms/step - loss: 1.0954 - categorical_accuracy: 0.5572 - val_loss: 0.8796 - val_categorical_accuracy: 0.6900

Epoch 00003: saving model to model_init_2018-10-2007_29_22.855379/model-00003-1.09540-0.55721-0.87963-0.69000.h5
Epoch 4/30
67/67 [==============================] - 14s 215ms/step - loss: 0.9611 - categorical_accuracy: 0.6269 - val_loss: 0.8779 - val_categorical_accuracy: 0.6900



67/67 [==============================] - 16s 236ms/step - loss: 1.0715 - categorical_accuracy: 0.5771 - val_loss: 0.8734 - val_categorical_accuracy: 0.7100

Epoch 00028: saving model to model_init_2018-10-2007_29_22.855379/model-00028-1.07152-0.57711-0.87341-0.71000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.525878978725359e-08.
Epoch 29/30
67/67 [==============================] - 16s 237ms/step - loss: 0.9801 - categorical_accuracy: 0.6020 - val_loss: 0.8736 - val_categorical_accuracy: 0.7100

Epoch 00029: saving model to model_init_2018-10-2007_29_22.855379/model-00029-0.98013-0.60199-0.87356-0.71000.h5
Epoch 30/30
67/67 [==============================] - 15s 220ms/step - loss: 0.9854 - categorical_accuracy: 0.5871 - val_loss: 0.8700 - val_categorical_accuracy: 0.7100

Epoch 00030: saving model to model_init_2018-10-2007_29_22.855379/model-00030-0.98538-0.58706-0.87003-0.71000.h5


In [96]:
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [123]:
def aug_generator(source_path, folder_list, batch_size):
      
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,nb_frames)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
  
            batch_data_aug2 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug2 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
 
            batch_data_aug3 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug3 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
      
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                M2 = get_random_affine()
                M3 = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes   
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    batch_data_aug2[folder,idx] = cv2.warpAffine(resized, M2, (resized.shape[0], resized.shape[1]))
                    batch_data_aug3[folder,idx] = cv2.warpAffine(resized, M3, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug2[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug3[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug2, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug3, axis = 0)
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug2, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug3, axis = 0)
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
  
            batch_data_aug2 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug2 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
   
            batch_data_aug3 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug3 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
      
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                M2 = get_random_affine()
                M3 = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    batch_data_aug2[folder,idx] = cv2.warpAffine(resized, M2, (resized.shape[0], resized.shape[1]))
                    batch_data_aug3[folder,idx] = cv2.warpAffine(resized, M3, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug2[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug3[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug2, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug3, axis = 0)
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug2, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug3, axis = 0) 
            yield batch_data, batch_labels
        

In [124]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_129 (Conv3D)          (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_59 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_129 (Activation)  (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_93 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_130 (Conv3D)          (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_60 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_130 (Activation)  (None, 15, 60, 60, 16)    0         
__________

In [127]:
batch_size = 5
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  ./Project_data/val ; batch size = 5
Source path =  ./Project_data/train ; batch size = 5
67/67 [==============================] - 26s 382ms/step - loss: 0.5547 - categorical_accuracy: 0.7769 - val_loss: 0.6688 - val_categorical_accuracy: 0.6550

Epoch 00001: saving model to model_init_2018-10-2007_29_22.855379/model-00001-0.55475-0.77687-0.66884-0.65500.h5
Epoch 2/20
67/67 [==============================] - 23s 347ms/step - loss: 0.5549 - categorical_accuracy: 0.7933 - val_loss: 0.8516 - val_categorical_accuracy: 0.7250

Epoch 00002: saving model to model_init_2018-10-2007_29_22.855379/model-00002-0.55736-0.79230-0.85163-0.72500.h5
Epoch 3/20
67/67 [==============================] - 14s 208ms/step - loss: 0.5180 - categorical_accuracy: 0.8085 - val_loss: 0.4881 - val_categorical_accuracy: 0.7900

Epoch 00003: saving model to model_init_2018-10-2007_29_22.855379/model-00003-0.51799-0.80846-0.48806-0.79000.h5
Epoch 4/20
67/67 [==============================] - 1

In [128]:
def aug_generator(source_path, folder_list, batch_size):
      
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,nb_frames)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
  
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes   
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 

            yield batch_data, batch_labels
        

In [129]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_133 (Conv3D)          (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_63 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_133 (Activation)  (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_97 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_134 (Conv3D)          (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_64 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_134 (Activation)  (None, 15, 60, 60, 16)    0         
__________

In [132]:
batch_size = 10
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  ./Project_data/train ; batch size = 10
Source path =  ./Project_data/val ; batch size = 10
67/67 [==============================] - 34s 503ms/step - loss: 0.3006 - categorical_accuracy: 0.8992 - val_loss: 0.5441 - val_categorical_accuracy: 0.8100

Epoch 00001: saving model to model_init_2018-10-2007_29_22.855379/model-00001-0.30186-0.89819-0.54408-0.81000.h5
Epoch 2/20
67/67 [==============================] - 10s 145ms/step - loss: 0.3312 - categorical_accuracy: 0.8781 - val_loss: 0.5516 - val_categorical_accuracy: 0.8050

Epoch 00002: saving model to model_init_2018-10-2007_29_22.855379/model-00002-0.33119-0.87811-0.55165-0.80500.h5
Epoch 3/20
67/67 [==============================] - 11s 158ms/step - loss: 0.4095 - categorical_accuracy: 0.8483 - val_loss: 0.5543 - val_categorical_accuracy: 0.8150

Epoch 00003: saving model to model_init_2018-10-2007_29_22.855379/model-00003-0.40951-0.84826-0.55427-0.81500.h5

Epoch 00003: ReduceLROnPlateau reducing learning r

In [135]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_141 (Conv3D)          (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_70 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_141 (Activation)  (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_105 (MaxPoolin (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_142 (Conv3D)          (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_71 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_142 (Activation)  (None, 15, 60, 60, 16)    0         
__________

In [136]:
batch_size = 10
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
67/67 [==============================] - 34s 512ms/step - loss: 1.6505 - categorical_accuracy: 0.3855 - val_loss: 1.2632 - val_categorical_accuracy: 0.3550

Epoch 00001: saving model to model_init_2018-10-2007_29_22.855379/model-00001-1.65914-0.38084-1.26319-0.35500.h5
Epoch 2/20
67/67 [==============================] - 9s 140ms/step - loss: 1.4908 - categorical_accuracy: 0.3856 - val_loss: 4.0222 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2018-10-2007_29_22.855379/model-00002-1.49079-0.38557-4.02221-0.23000.h5
Epoch 3/20
67/67 [==============================] - 11s 162ms/step - loss: 1.5770 - categorical_accuracy: 0.3035 - val_loss: 1.4175 - val_categorical_accuracy: 0.3050

Epoch 00003: saving model to model_init_2018-10-2007_29_22.855379/model-00003-1.57701-0.30348-1.41755-0.30500.h5

Epoch 00003: ReduceLROnPlateau reducing learning ra